*Note: You are currently reading this using Google Colaboratory which is a cloud-hosted version of Jupyter Notebook. This is a document containing both text cells for documentation and runnable code cells. If you are unfamiliar with Jupyter Notebook, watch this 3-minute introduction before starting this challenge: https://www.youtube.com/watch?v=inN8seMm7UI*

---

In this challenge, you will create a book recommendation algorithm using **K-Nearest Neighbors**.

You will use the [Book-Crossings dataset](http://www2.informatik.uni-freiburg.de/~cziegler/BX/). This dataset contains 1.1 million ratings (scale of 1-10) of 270,000 books by 90,000 users. 

After importing and cleaning the data, use `NearestNeighbors` from `sklearn.neighbors` to develop a model that shows books that are similar to a given book. The Nearest Neighbors algorithm measures distance to determine the “closeness” of instances.

Create a function named `get_recommends` that takes a book title (from the dataset) as an argument and returns a list of 5 similar books with their distances from the book argument.

This code:

`get_recommends("The Queen of the Damned (Vampire Chronicles (Paperback))")`

should return:

```
[
  'The Queen of the Damned (Vampire Chronicles (Paperback))',
  [
    ['Catch 22', 0.793983519077301], 
    ['The Witching Hour (Lives of the Mayfair Witches)', 0.7448656558990479], 
    ['Interview with the Vampire', 0.7345068454742432],
    ['The Tale of the Body Thief (Vampire Chronicles (Paperback))', 0.5376338362693787],
    ['The Vampire Lestat (Vampire Chronicles, Book II)', 0.5178412199020386]
  ]
]
```

Notice that the data returned from `get_recommends()` is a list. The first element in the list is the book title passed in to the function. The second element in the list is a list of five more lists. Each of the five lists contains a recommended book and the distance from the recommended book to the book passed in to the function.

If you graph the dataset (optional), you will notice that most books are not rated frequently. To ensure statistical significance, remove from the dataset users with less than 200 ratings and books with less than 100 ratings.

The first three cells import libraries you may need and the data to use. The final cell is for testing. Write all your code in between those cells.

# Load libraries

In [ ]:
# import libraries (you may add additional imports but you may not have to)
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt

# Load dataset

In [ ]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/books/book-crossings.zip

!unzip book-crossings.zip

books_filename = 'BX-Books.csv'
ratings_filename = 'BX-Book-Ratings.csv'

--2022-08-08 16:12:53--  https://cdn.freecodecamp.org/project-data/books/book-crossings.zip
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 104.26.2.33, 172.67.70.149, 104.26.3.33, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|104.26.2.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 26085508 (25M) [application/zip]
Saving to: ‘book-crossings.zip.3’

book-crossings.zip. 100%[===================>]  24.88M  92.0MB/s    in 0.3s    

2022-08-08 16:12:54 (92.0 MB/s) - ‘book-crossings.zip.3’ saved [26085508/26085508]

Archive:  book-crossings.zip
replace BX-Book-Ratings.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: BX-Book-Ratings.csv     
replace BX-Books.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: BX-Books.csv            
replace BX-Users.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: BX-Users.csv            


# Preprocess data to make a review table

In [ ]:
# import csv data into dataframes
df_books = pd.read_csv(
    books_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['isbn', 'title', 'author'],
    usecols=['isbn', 'title', 'author'],
    dtype={'isbn': 'str', 'title': 'str', 'author': 'str'})

df_ratings = pd.read_csv(
    ratings_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['user', 'isbn', 'rating'],
    usecols=['user', 'isbn', 'rating'],
    dtype={'user': 'int32', 'isbn': 'str', 'rating': 'float32'})

In [ ]:
df_books

,isbn,title,author
0,0195153448,Classical Mythology,Mark P. O. Morford
1,0002005018,Clara Callan,Richard Bruce Wright
2,0060973129,Decision in Normandy,Carlo D'Este
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata
4,0393045218,The Mummies of Urumchi,E. J. W. Barber
...,...,...,...
271374,0440400988,There's a Bat in Bunk Five,Paula Danziger
271375,0525447644,From One to One Hundred,Teri Sloat
271376,006008667X,Lily Dale : The True Story of the Town that Ta...,Christine Wicker
271377,0192126040,Republic (World's Classics),Plato


In [ ]:
df_ratings

,user,isbn,rating
0,276725,034545104X,0.0
1,276726,0155061224,5.0
2,276727,0446520802,0.0
3,276729,052165615X,3.0
4,276729,0521795028,6.0
...,...,...,...
1149775,276704,1563526298,9.0
1149776,276706,0679447156,0.0
1149777,276709,0515107662,10.0
1149778,276721,0590442449,10.0


In [ ]:
df_ratings.isnull().sum()

user      0
isbn      0
rating    0
dtype: int64

In [ ]:
df_books.isnull().sum()

isbn      0
title     0
author    1
dtype: int64

In [ ]:
df_books.dropna(inplace=True)

In [ ]:
df_books.isnull().sum()

isbn      0
title     0
author    0
dtype: int64

In [ ]:
# remove users with less than 200 ratings
# remove users that appear in the user column less than 200 times

In [ ]:
# number of unique users
df_unique_users = df_ratings['user'].value_counts().rename_axis('user_id').reset_index(name='number_of_ratings')
len(df_unique_users)

105283

In [ ]:
# unique users
# how many times they appear in the user column = how many ratings they have made
df_unique_users

,user_id,number_of_ratings
0,11676,13602
1,198711,7550
2,153662,6109
3,98391,5891
4,35859,5850
...,...,...
105278,116180,1
105279,116166,1
105280,116154,1
105281,116137,1


In [ ]:
# number of unique users with ratings < 200, we want to remove them
df_unique_users['number_of_ratings'][df_unique_users['number_of_ratings'] < 200].count()

104378

In [ ]:
df_users_to_be_removed = df_unique_users['number_of_ratings'][df_unique_users['number_of_ratings'] < 200].rename_axis('user_id').reset_index(name='number_of_ratings < 200')
df_users_to_be_removed

,user_id,number_of_ratings < 200
0,905,199
1,906,199
2,907,199
3,908,198
4,909,198
...,...,...
104373,105278,1
104374,105279,1
104375,105280,1
104376,105281,1


In [ ]:
# we need series for isin(series), not dataframe column
unique_users = df_ratings['user'].value_counts()
users_to_be_removed = unique_users[unique_users < 200]
users_to_be_removed

193458    199
240403    199
203017    199
79942     198
267061    198
         ... 
116180      1
116166      1
116154      1
116137      1
276723      1
Name: user, Length: 104378, dtype: int64

In [ ]:
# number of rows we want to remove
remove = df_ratings['user'].isin(users_to_be_removed.index)
remove.sum()

622224

In [ ]:
remove

0          True
1          True
2          True
3          True
4          True
           ... 
1149775    True
1149776    True
1149777    True
1149778    True
1149779    True
Name: user, Length: 1149780, dtype: bool

In [ ]:
# number of users - number of users with ratings < 200 = number of user ratings >= 200
users_cleaned = df_ratings[~remove]
users_cleaned.shape

(527556, 3)

In [ ]:
users_cleaned

,user,isbn,rating
1456,277427,002542730X,10.0
1457,277427,0026217457,0.0
1458,277427,003008685X,8.0
1459,277427,0030615321,0.0
1460,277427,0060002050,0.0
...,...,...,...
1147612,275970,3829021860,0.0
1147613,275970,4770019572,0.0
1147614,275970,896086097,0.0
1147615,275970,9626340762,8.0


In [ ]:
df_ratings_cleaned = df_ratings[~remove]
df_ratings_cleaned

,user,isbn,rating
1456,277427,002542730X,10.0
1457,277427,0026217457,0.0
1458,277427,003008685X,8.0
1459,277427,0030615321,0.0
1460,277427,0060002050,0.0
...,...,...,...
1147612,275970,3829021860,0.0
1147613,275970,4770019572,0.0
1147614,275970,896086097,0.0
1147615,275970,9626340762,8.0


In [ ]:
# remove books with less than 100 ratings
# remove book that appear in the isbn column less than 100 times

In [ ]:
# number of unique books
df_unique_books = df_ratings['isbn'].value_counts().rename_axis('book_id').reset_index(name='number_of_ratings')
len(df_unique_books)

340556

In [ ]:
# unique books
# how many times they appear in the user column = how many ratings they have
df_unique_books

,book_id,number_of_ratings
0,0971880107,2502
1,0316666343,1295
2,0385504209,883
3,0060928336,732
4,0312195516,723
...,...,...
340551,1568656386,1
340552,1568656408,1
340553,1569551553,1
340554,1570081808,1


In [ ]:
# number of unique books with ratings < 100, we want to remove them
df_unique_books['number_of_ratings'][df_unique_books['number_of_ratings'] < 100].count()

339825

In [ ]:
df_books_to_be_removed = df_unique_books['number_of_ratings'][df_unique_books['number_of_ratings'] < 100].rename_axis('book_id').reset_index(name='number_of_ratings < 200')
df_books_to_be_removed

,book_id,number_of_ratings < 200
0,731,99
1,732,99
2,733,99
3,734,99
4,735,99
...,...,...
339820,340551,1
339821,340552,1
339822,340553,1
339823,340554,1


In [ ]:
# we need series for isin(series), not dataframe column
unique_books = df_ratings['isbn'].value_counts()
books_to_be_removed = unique_books[unique_books < 100]
books_to_be_removed

0375500510     99
0671727583     99
0425174271     99
1576737330     99
0425172996     99
               ..
1568656386      1
1568656408      1
1569551553      1
1570081808      1
05162443314     1
Name: isbn, Length: 339825, dtype: int64

In [ ]:
# number of rows we want to remove
remove_from_ratings_cleaned = df_ratings_cleaned['isbn'].isin(books_to_be_removed.index)
remove.sum()

622224

In [ ]:
ratings_cleaned = df_ratings_cleaned[~remove_from_ratings_cleaned] 
ratings_cleaned.shape

(49781, 3)

In [ ]:
ratings_cleaned

,user,isbn,rating
1456,277427,002542730X,10.0
1469,277427,0060930535,0.0
1471,277427,0060934417,0.0
1474,277427,0061009059,9.0
1484,277427,0140067477,0.0
...,...,...,...
1147304,275970,0804111359,0.0
1147436,275970,140003065X,0.0
1147439,275970,1400031346,0.0
1147440,275970,1400031354,0.0


In [ ]:
# number of rows we want to remove
remove_from_books = df_books['isbn'].isin(books_to_be_removed.index)
remove_from_books.sum()

269442

In [ ]:
books_cleaned = df_books[~remove_from_books]
books_cleaned.shape

(1936, 3)

In [ ]:
books_cleaned

,isbn,title,author
18,0440234743,The Testament,John Grisham
19,0452264464,Beloved (Plume Contemporary Fiction),Toni Morrison
26,0971880107,Wild Animus,Rich Shapero
27,0345402871,Airframe,Michael Crichton
28,0345417623,Timeline,MICHAEL CRICHTON
...,...,...,...
271360,1874166633,Introducing Nietzsche (Foundations in Children...,Laurence Gane
271361,0130897930,Core Web Programming (2nd Edition),Marty Hall
271362,020130998X,The Unified Modeling Language Reference Manual...,James Rumbaugh
271363,2268032019,Petite histoire de la dÃ?Â©sinformation,Vladimir Volkoff


In [ ]:
user_book_table = ratings_cleaned.pivot_table(index=['user'], columns=['isbn'], values='rating').fillna(0).T
user_book_table.head()

user,254,2276,2766,2977,3363,4017,4385,6242,6251,6323,...,274004,274061,274301,274308,274808,275970,277427,277478,277639,278418
isbn,,,,,,,,,,,,,,,,,,,,,
002542730X,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,10.0,0.0,0.0,0.0
0060008032,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0060096195,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
006016848X,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0060173289,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
user_book_table.index = user_book_table.join(df_books.set_index('isbn'))['title']
user_book_table.head()

user,254,2276,2766,2977,3363,4017,4385,6242,6251,6323,...,274004,274061,274301,274308,274808,275970,277427,277478,277639,278418
title,,,,,,,,,,,,,,,,,,,,,
Politically Correct Bedtime Stories: Modern Tales for Our Life and Times,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,10.0,0.0,0.0,0.0
Angels,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
The Boy Next Door,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"Men Are from Mars, Women Are from Venus: A Practical Guide for Improving Communication and Getting What You Want in Your Relationships",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Divine Secrets of the Ya-Ya Sisterhood : A Novel,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
user_book_table = user_book_table.sort_index()
user_book_table.head(15)

user,254,2276,2766,2977,3363,4017,4385,6242,6251,6323,...,274004,274061,274301,274308,274808,275970,277427,277478,277639,278418
title,,,,,,,,,,,,,,,,,,,,,
1984,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1st to Die: A Novel,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1st to Die: A Novel,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2nd Chance,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2nd Chance,0.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4 Blondes,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A Beautiful Mind: The Life of Mathematical Genius and Nobel Laureate John Nash,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A Bend in the Road,0.0,0.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A Case of Need,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
number_of_books = user_book_table.shape[0] # number of books = rows of the table
number_of_books

731

In [ ]:
number_of_users = user_book_table.shape[1] # number of users = columns of the table
number_of_users

888

In [ ]:
# We choose random book
query_index = np.random.choice(number_of_books)
title = user_book_table.index[query_index]
print("Number of row of the table is:", query_index) 
print("Choosen book is:", title)

Number of row of the table is: 424
Choosen book is: Shopaholic Takes Manhattan (Summer Display Opportunity)


In [ ]:
user_book_table.loc[title] # ratings of the users, we get the row in which the book title is, 1st way

user
254       0.0
2276      0.0
2766      0.0
2977      0.0
3363      0.0
         ... 
275970    0.0
277427    0.0
277478    0.0
277639    0.0
278418    0.0
Name: Shopaholic Takes Manhattan (Summer Display Opportunity), Length: 888, dtype: float32

In [ ]:
user_book_table.iloc[query_index, :] # ratings of the users, we get the row in which the book title is, 2nd way

user
254       0.0
2276      0.0
2766      0.0
2977      0.0
3363      0.0
         ... 
275970    0.0
277427    0.0
277478    0.0
277639    0.0
278418    0.0
Name: Shopaholic Takes Manhattan (Summer Display Opportunity), Length: 888, dtype: float32

In [ ]:
ratings = user_book_table.iloc[query_index, :].values # 1D Array
ratings

array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  8.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  8.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  6.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  8.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  8.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  8.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0

In [ ]:
ratings.shape

(888,)

In [ ]:
ratings = np.reshape(ratings, (1, -1)) # 2D Array
ratings.shape

(1, 888)

In [ ]:
user_book_table_matrix = csr_matrix(user_book_table.values)

# KNearestNeighbors

In [ ]:
# we will use cosine similarity for nearest neighbor search, not euclidean distance
model_knn = NearestNeighbors(metric='cosine')
model_knn.fit(user_book_table_matrix)

NearestNeighbors(metric='cosine')

In [ ]:
distances, indices = model_knn.kneighbors(ratings, n_neighbors = 6)

In [ ]:
print(distances)
print(indices)

[[1.7881393e-07 6.0665953e-01 6.2896931e-01 6.3073683e-01 7.4943924e-01
  7.6400834e-01]]
[[424 425 517 127 587 222]]


In [ ]:
indices.shape

(1, 6)

In [ ]:
# we get 1st list and all the five numbers
user_book_table.iloc[indices[0]].index.values

array(['Shopaholic Takes Manhattan (Summer Display Opportunity)',
       'Shopaholic Ties the Knot', 'The Devil Wears Prada : A Novel',
       'Confessions of a Shopaholic (Summer Display Opportunity)',
       'The Nanny Diaries: A Novel', 'Girl in Hyacinth Blue'],
      dtype=object)

In [ ]:
user_book_table.iloc[indices[0]].index # index of rows of the table

Index(['Shopaholic Takes Manhattan (Summer Display Opportunity)',
       'Shopaholic Ties the Knot', 'The Devil Wears Prada : A Novel',
       'Confessions of a Shopaholic (Summer Display Opportunity)',
       'The Nanny Diaries: A Novel', 'Girl in Hyacinth Blue'],
      dtype='object', name='title')

In [ ]:
df = pd.DataFrame({
    'title'   : user_book_table.iloc[indices[0]].index.values,
    'distance': distances[0]
})

df

,title,distance
0,Shopaholic Takes Manhattan (Summer Display Opp...,1.788139e-07
1,Shopaholic Ties the Knot,6.066595e-01
2,The Devil Wears Prada : A Novel,6.289693e-01
3,Confessions of a Shopaholic (Summer Display Op...,6.307368e-01
4,The Nanny Diaries: A Novel,7.494392e-01
5,Girl in Hyacinth Blue,7.640083e-01


In [ ]:
df.sort_values(by='distance', ascending=False)

,title,distance
5,Girl in Hyacinth Blue,7.640083e-01
4,The Nanny Diaries: A Novel,7.494392e-01
3,Confessions of a Shopaholic (Summer Display Op...,6.307368e-01
2,The Devil Wears Prada : A Novel,6.289693e-01
1,Shopaholic Ties the Knot,6.066595e-01
0,Shopaholic Takes Manhattan (Summer Display Opp...,1.788139e-07


In [ ]:
df.values.tolist()

[['Shopaholic Takes Manhattan (Summer Display Opportunity)',
  1.7881393432617188e-07],
 ['Shopaholic Ties the Knot', 0.6066595315933228],
 ['The Devil Wears Prada : A Novel', 0.6289693117141724],
 ['Confessions of a Shopaholic (Summer Display Opportunity)',
  0.6307368278503418],
 ['The Nanny Diaries: A Novel', 0.7494392395019531],
 ['Girl in Hyacinth Blue', 0.7640083432197571]]

# Testing

In [ ]:
# function to return recommended books - this will be tested
def get_recommends(book = ""):
  # book = user_book_table.loc[book]
  # ratings = user_book_table.loc[book].values # 1D Array
  # ratings = np.reshape(ratings, (1, -1)) # 2D Array
  ratings = user_book_table.loc[book].values.reshape(1, -1)

  distances, indices = model_knn.kneighbors(ratings, n_neighbors = 6)
  df = pd.DataFrame({
    'title'   : user_book_table.iloc[indices[0]].index.values,
    'distance': distances[0]
  })

  similar_books = df.values.tolist()
  similar_books.pop(0)
  similar_books.reverse()
  # title = recommended_books.append(df.iloc[0].at['title'])
  recommended_books = [book, similar_books]
  return recommended_books

Use the cell below to test your function. The `test_book_recommendation()` function will inform you if you passed the challenge or need to keep trying.

In [ ]:
books = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
print(books)

def test_book_recommendation():
  test_pass = True
  recommends = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
  if recommends[0] != "Where the Heart Is (Oprah's Book Club (Paperback))":
    test_pass = False
  recommended_books = ["I'll Be Seeing You", 'The Weight of Water', 'The Surgeon', 'I Know This Much Is True', 'The Lovely Bones: A Novel']
  recommended_books_dist = [0.8, 0.77, 0.77, 0.77, 0.72]
  for i in range(2): 
    if recommends[1][i][0] not in recommended_books:
      test_pass = False
    if abs(recommends[1][i][1] - recommended_books_dist[i]) >= 0.05:
      test_pass = False
  if test_pass:
    print("You passed the challenge! 🎉🎉🎉🎉🎉")
  else:
    print("You haven't passed yet. Keep trying!")

test_book_recommendation()

["Where the Heart Is (Oprah's Book Club (Paperback))", [["I'll Be Seeing You", 0.8016210794448853], ['The Weight of Water', 0.7708583474159241], ['The Surgeon', 0.7699410915374756], ['I Know This Much Is True', 0.7677075266838074], ['The Lovely Bones: A Novel', 0.7234864234924316]]]
You passed the challenge! 🎉🎉🎉🎉🎉
